<a href="https://colab.research.google.com/github/Nusnan-AM/Research_models/blob/main/JAR_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ANN


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Input, Dropout  # Import Dropout
from sklearn.metrics import r2_score
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/jartest.csv')
df.head()

,Date,Raw_Turbidity,Raw_PH,Raw_Colour,PAC,KMnO4,ACD,Turbidity,PH,Colour
0,31/12/2019,90.4,7.07,500.0,2.5,0.4,0.5,19.0,7.07,48.0
1,31/12/2019,90.4,7.07,500.0,3.0,0.4,0.5,14.2,7.05,39.0
2,31/12/2019,90.4,7.07,500.0,3.5,0.4,0.5,12.6,7.02,31.0
3,31/12/2019,90.4,7.07,500.0,4.0,0.4,0.5,10.6,6.98,27.0
4,31/12/2019,90.4,7.07,500.0,4.5,0.4,0.5,10.7,6.93,29.0


In [ ]:
input_features = ["Raw_Turbidity", "Raw_PH", "Raw_Colour", "PAC", "KMnO4", "ACD"]
target_variables = ["Turbidity", "PH", "Colour"]  # Multi-output regression

X = df[input_features]
Y = df[target_variables]  # Multiple outputs

# Split data into train & test sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalize features and target variables
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
Y_train = scaler_Y.fit_transform(Y_train)
Y_test = scaler_Y.transform(Y_test)

In [ ]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),  # More neurons
    BatchNormalization(),  # Normalize activations
    Dropout(0.3),  # Higher dropout to prevent overfitting
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(len(target_variables))  # Multi-output regression
])

# Compile Model with Lower Learning Rate
optimizer = Adam(learning_rate=0.001)  # Slower learning rate for stability
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(X_train, Y_train, epochs=200, batch_size=32, validation_data=(X_test, Y_test), verbose=2)

Epoch 1/200
128/128 - 4s - 32ms/step - loss: 0.3326 - mae: 0.4190 - val_loss: 0.0507 - val_mae: 0.1629
Epoch 2/200
128/128 - 1s - 8ms/step - loss: 0.1029 - mae: 0.2452 - val_loss: 0.0280 - val_mae: 0.1220
Epoch 3/200
128/128 - 1s - 4ms/step - loss: 0.0644 - mae: 0.1924 - val_loss: 0.0213 - val_mae: 0.1098
Epoch 4/200
128/128 - 1s - 5ms/step - loss: 0.0435 - mae: 0.1581 - val_loss: 0.0180 - val_mae: 0.1024
Epoch 5/200
128/128 - 1s - 4ms/step - loss: 0.0345 - mae: 0.1396 - val_loss: 0.0159 - val_mae: 0.0894
Epoch 6/200
128/128 - 1s - 4ms/step - loss: 0.0283 - mae: 0.1238 - val_loss: 0.0163 - val_mae: 0.0923
Epoch 7/200
128/128 - 1s - 5ms/step - loss: 0.0248 - mae: 0.1154 - val_loss: 0.0164 - val_mae: 0.0909
Epoch 8/200
128/128 - 1s - 4ms/step - loss: 0.0226 - mae: 0.1082 - val_loss: 0.0145 - val_mae: 0.0844
Epoch 9/200
128/128 - 1s - 5ms/step - loss: 0.0198 - mae: 0.1005 - val_loss: 0.0138 - val_mae: 0.0791
Epoch 10/200
128/128 - 1s - 10ms/step - loss: 0.0189 - mae: 0.0982 - val_loss: 0.

In [ ]:
Y_pred = model.predict(X_test)

# Inverse transform predictions to original scale
Y_pred_original = scaler_Y.inverse_transform(Y_pred)
Y_test_original = scaler_Y.inverse_transform(Y_test)

# Evaluate performance for each target variable
for i, target in enumerate(target_variables):
    mae = mean_absolute_error(Y_test_original[:, i], Y_pred_original[:, i])
    mse = mean_squared_error(Y_test_original[:, i], Y_pred_original[:, i])
    r2 = r2_score(Y_test_original[:, i], Y_pred_original[:, i])

    print(f"\n📊 Performance for {target}:")
    print(f"   MAE: {mae:.2f}")
    print(f"   MSE: {mse:.2f}")
    print(f"   R²: {r2:.2f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

📊 Performance for Turbidity:
   MAE: 5.87
   MSE: 71.38
   R²: 0.44

📊 Performance for PH:
   MAE: 0.10
   MSE: 0.03
   R²: 0.73

📊 Performance for Colour:
   MAE: 19.09
   MSE: 1134.21
   R²: 0.42
